## 0. Python imports & setup

For learning purposes, libraries will be imported inside its corresponding usage section...

## 1. Data loading

In [103]:
import pandas as pd

* incidents: labeled data we can use for training and testing

In [104]:
incidents = pd.read_csv('../data/processed/pipelines_incident.csv')

as you can see, there are both categorical and numerical columns...

In [105]:
incidents.drop(columns={'Unnamed: 0'}, inplace=True)

## 2. Data Exploration

Although the exploration of the dataset was made in the notebook data_analysis_report, we are going to make sure that there is no null values(scikit-learn does not goes well with nulls...)

In [106]:
incidents.isna().sum()

LOCAL_DATETIME                  0
REPORT_NUMBER                   0
SIGNIFICANT                     0
SERIOUS                         0
ON_OFF_SHORE                    0
LOCATION_CITY_NAME             94
LOCATION_COUNTY_NAME          239
LOCATION_STATE                  0
INCIDENT_AREA_TYPE              0
CAUSE                           0
MAP_CAUSE                       0
MAP_SUBCAUSE                    0
FATAL                           0
INJURE                          0
UNINTENTIONAL_RELEASE_BBLS      0
TOTAL_COST_IN84                 0
TOTAL_COST_CURRENT              0
ACCIDENT_PSIG                   0
MOP_PSIG                        0
OPERATOR_NAME                   0
OPERATOR_ID                     0
MATERIAL_INVOLVED               0
COMMODITY_RELEASED_TYPE         0
dtype: int64

In [108]:
incidents[['LOCATION_CITY_NAME', 'LOCATION_COUNTY_NAME']].fillna('NONE', inplace=True)

See what were the types of the categorical and numerical columns

In [109]:
incidents.dtypes

LOCAL_DATETIME                 object
REPORT_NUMBER                   int64
SIGNIFICANT                    object
SERIOUS                        object
ON_OFF_SHORE                   object
LOCATION_CITY_NAME             object
LOCATION_COUNTY_NAME           object
LOCATION_STATE                 object
INCIDENT_AREA_TYPE             object
CAUSE                          object
MAP_CAUSE                      object
MAP_SUBCAUSE                   object
FATAL                         float64
INJURE                        float64
UNINTENTIONAL_RELEASE_BBLS    float64
TOTAL_COST_IN84                 int64
TOTAL_COST_CURRENT              int64
ACCIDENT_PSIG                 float64
MOP_PSIG                      float64
OPERATOR_NAME                  object
OPERATOR_ID                     int64
MATERIAL_INVOLVED              object
COMMODITY_RELEASED_TYPE        object
dtype: object

## 3. Preprocessing

In this section we will see how to use scikit-learn's Pipiline and ColumnTransformer, one of the best practices for composing preprocessing and modeling in a single and elegand class... pay attention as it is hard to understand...

In [110]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

* https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
* https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

let's identify numerical and categorical features...

In [111]:
CAT_FEATS = ['SERIOUS', 'ON_OFF_SHORE', 'LOCATION_STATE','INCIDENT_AREA_TYPE', 'CAUSE','MAP_CAUSE', 'OPERATOR_NAME', 'MATERIAL_INVOLVED', 'COMMODITY_RELEASED_TYPE']
NUM_FEATS = ['REPORT_NUMBER', 'FATAL', 'INJURE', 'UNINTENTIONAL_RELEASE_BBLS', 'ACCIDENT_PSIG', 'MOP_PSIG']
FEATS = NUM_FEATS + CAT_FEATS
TARGET = 'TOTAL_COST_CURRENT'

let's define a preprocessing transformer for numerical columns...

In [112]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), 
                ('scaler', StandardScaler())])

let's define a preprocessing transformer for categorical columns...

In [113]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

let's join these transformers using a `ColumnTransformer`:

In [114]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                                ('cat', categorical_transformer, CAT_FEATS)])

In [115]:
preprocessor.fit_transform(incidents)

<18692x2725 sparse matrix of type '<class 'numpy.float64'>'
	with 280380 stored elements in Compressed Sparse Row format>

Since we have obtained a sparse matrix, we need to convert the data with the method "to dense"

In [116]:
preprocessor.fit_transform(incidents).todense()

matrix([[-1.8949585 ,  2.5152589 , -0.02613896, ...,  0.        ,
          0.        ,  0.        ],
        [-1.89459727, -0.09076179, -0.02613896, ...,  0.        ,
          0.        ,  0.        ],
        [-1.89505884, -0.09076179,  0.07779412, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 1.61713354, -0.09076179, -0.02613896, ...,  0.        ,
          0.        ,  0.        ],
        [ 1.61691279, -0.09076179, -0.02613896, ...,  0.        ,
          0.        ,  0.        ],
        [ 1.61685259, -0.09076179, -0.02613896, ...,  0.        ,
          0.        ,  0.        ]])

inspecting the full preprocessor:

at least in this case, it is at the cost of interpretability of transformed DataFrame...

In [117]:
pd.DataFrame(data=preprocessor.fit_transform(incidents).todense()).head()

,0,1,2,3,4,5,6,7,8,9,...,2715,2716,2717,2718,2719,2720,2721,2722,2723,2724
0,-1.894958,2.515259,-0.026139,-0.127886,-0.660712,-0.639399,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.894597,-0.090762,-0.026139,-0.100576,-0.457009,-0.640322,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.895059,-0.090762,0.077794,-0.127886,-0.597582,-0.529577,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.894577,-0.090762,-0.026139,-0.127886,-0.541352,-0.548035,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.894878,-0.090762,0.077794,-0.127886,-0.660712,-0.640322,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4. Train a simple model

First, lets train a simple model using holdout, train - test split...

In [118]:
from sklearn.model_selection import train_test_split

In [119]:
incidents_train, incidents_test = train_test_split(incidents)

In [120]:
print(incidents_train.shape)
print(incidents_test.shape)

(14019, 23)
(4673, 23)


let's choose a model from scikit-learn cheatsheet: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [121]:
from sklearn.ensemble import RandomForestRegressor

rfr_model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', RandomForestRegressor())])

In [122]:
rfr_model.fit(incidents_train[FEATS], incidents_train[TARGET]);

### 4.1 train a LightGBM model

In [65]:
from lightgbm import LGBMRegressor

lgbm_model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', LGBMRegressor())])

In [66]:
lgbm_model.fit(incidents_train[FEATS], incidents_train[TARGET]);

### 4.2 train a combined model

In [70]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

r1 = LGBMRegressor()
r2 = RandomForestRegressor()

combined_model= Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', VotingRegressor([('lgbm', r1), ('rf', r2)]))])

In [91]:
combined_model.fit(incidents_train[FEATS], incidents_train[TARGET]);

KeyboardInterrupt: 

## 5. check model performance on test and train data

In [123]:
from sklearn.metrics import mean_squared_error

In [124]:
y_test = rfr_model.predict(incidents_test[FEATS])
y_train = rfr_model.predict(incidents_train[FEATS])

In [127]:
print(f"test error: {mean_squared_error(y_pred=y_test, y_true=incidents_test[TARGET], squared=False)}")
print(f"train error: {mean_squared_error(y_pred=y_train, y_true=incidents_train[TARGET], squared=False)}")

test error: 15071583.884057237
train error: 6805407.022788092


## 6. check model performance using cross validation

In [128]:
from sklearn.model_selection import cross_val_score

In [129]:
scores = cross_val_score(rfr_model, 
                         incidents[FEATS], 
                         incidents[TARGET], 
                         scoring='neg_root_mean_squared_error', 
                         cv=5, n_jobs=-1)

KeyboardInterrupt: 

In [ ]:
import numpy as np
np.mean(-scores)

## 7. optimize model using grid search

In [88]:
from sklearn.model_selection import RandomizedSearchCV

In [204]:
lgbm_param_grid = {'regressor__num_leaves': (20, 100),
                   'regressor__n_estimators': (20, 500),
                   'regressor__learning_rate': (0.05, 0.3),
                  'regressor__feature_fraction': (0.1, 0.9),
                  'regressor__bagging_fraction': (0.8, 1),
                  'regressor__max_depth': (15, 25),
                  'regressor__min_split_gain': (0.001, 0.1),
                  'regressor__min_child_weight': (10, 50),
                'regressor__preprocessor__num__imputer__strategy': ['mean', 'median']}

rfr_param_grid = {'regressor__n_estimators': [512],
                 'regressor__max_depth': [16],
                 'preprocessor__num__imputer__strategy': ['mean']}

combined_param_grid = {
                        'regressor__lgbm__num_leaves': (20, 100),
                        'regressor__lgbm__n_estimators': (20, 500),
                        'regressor__lgbm__learning_rate': (0.05, 0.3),
                        'regressor__lgbm__feature_fraction': (0.1, 0.9),
                        'regressor__lgbm__bagging_fraction': (0.8, 1),
                        'regressor__lgbm__max_depth': (15, 25),
                        'regressor__lgbm__min_split_gain': (0.001, 0.1),
                        'regressor__lgbm__min_child_weight': (10, 50),
                        'regressor__rf__n_estimators': [512],
                        'regressor__rf__max_depth': [16],
                        'preprocessor__num__imputer__strategy': ['mean']
}


grid_search = RandomizedSearchCV(combined_model, 
                                 combined_param_grid, 
                                 cv=5, 
                                 verbose=2, 
                                 scoring='neg_root_mean_squared_error', 
                                 n_jobs=-1,
                                 n_iter=10)

grid_search.fit(diamonds[FEATS], diamonds[TARGET])

/home/elo/miniconda3/envs/ironhack_env/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..........preprocessor__num__imputer__strategy=mean; total time= 1.7min
[CV] END ..........preprocessor__num__imputer__strategy=mean; total time= 1.9min
[CV] END ..........preprocessor__num__imputer__strategy=mean; total time= 1.6min
[CV] END ..........preprocessor__num__imputer__strategy=mean; total time= 1.6min
[CV] END ..........preprocessor__num__imputer__strategy=mean; total time= 1.7min


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'depth',
                                                                                'table',
                                                                                'x',
              

In [206]:
grid_search.best_params_

{'preprocessor__num__imputer__strategy': 'mean'}

In [207]:
grid_search.best_score_

-534.2560032819135

## 8. Prepare submission

In [208]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

In [209]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'], 'price': y_pred})

In [210]:
submission_df.head()

,id,price
0,0,2907.639331
1,1,5632.615022
2,2,9471.212337
3,3,4055.583619
4,4,1639.378447


In [211]:
submission_df.describe()

,id,price
count,13485.000000,13485.000000
mean,6742.000000,3952.955395
std,3892.928525,3945.681812
min,0.000000,339.102748
25%,3371.000000,945.682904
50%,6742.000000,2466.956301
75%,10113.000000,5304.791594
max,13484.000000,17899.397882


In [212]:
submission_df.price.clip(0, 20000, inplace=True)

In [213]:
submission_df.to_csv('../submissions_kaggle//submission_combined_3.csv', index=False)